In [1]:
from fase import HEAAN
import fase.HEAAN as he
import numpy as np

In [2]:
class Param():
    def __init__(self, n=None, logn=None, logp=None, logq=None, logQboot=None):
        self.n = n
        self.logn = logn
        self.logp = logp
        self.logq = logq 
        self.logQboot = logQboot
        if self.logn == None:
            self.logn = int(np.log2(n))

In [3]:
logq = 540
logp = 30
logn = 14
n = 1*2**logn
slots = n

parms = Param(n=n, logp=logp, logq=logq)

do_reduction=False
is_serialized = True

ring = he.Ring()

In [13]:
secretKey = he.SecretKey(ring)
#he.SerializationUtils.writeKey(secretKey., "./test/serkey/SECRETKEY.txt")
scheme = he.Scheme(secretKey, ring, False, './test/') # Failed to store secret key...  it's not working.

algo = he.SchemeAlgo(scheme)

# reduction때는 right rotation N_class개 필요. 
if do_reduction:
    Nclass = Nmodel.head.shape[0]
    scheme.addLeftRotKeys(secretKey)
    for i in range(Nclass):
        scheme.addRightRotKey(secretKey, i+1) # 
else:
    # reduction 안 하면 하나짜리 rotation만 여러번 반복.
    scheme.addLeftRotKey(secretKey, 1)

In [4]:
scheme2 = he.Scheme(ring, is_serialized, './test/')

loading Keys from ./test/serkey/ENCRYPTION.txt
loading Keys from ./test/serkey/MULTIPLICATION.txt


In [5]:
def decrypt(scheme, secretKey, enc):
    featurized = scheme.decrypt(secretKey, enc)
    arr = np.zeros(n, dtype=np.complex128)
    featurized.__getarr__(arr)
    return arr.real

def encrypt(scheme, val):
    ctxt = he.Ciphertext()#logp, logq, n)
    vv = np.zeros(n) # Need to initialize to zero or will cause "unbound"
    vv[:len(val)] = val
    scheme.encrypt(ctxt, he.Double(vv), n, logp, logq)
    del vv
    return ctxt

In [7]:
ctx1 = encrypt(scheme, [1,2,3,4,5,6,7])
ctx1_1 = encrypt(scheme, [1,2,3,4,5,6,7])
ctx2 = encrypt(scheme, [4,5,6,7,8,9,0])


In [9]:
he.SerializationUtils.writeCiphertext(ctx1, "./ctx1.dat")
he.SerializationUtils.writeCiphertext(ctx1_1, "./ctx1_1.dat")
he.SerializationUtils.writeCiphertext(ctx2, "./ctx2.dat")

In [6]:
ctx1 = he.Ciphertext()
he.SerializationUtils.readCiphertext(ctx1, "./ctx1.dat")

In [8]:
decrypt(ctx1)

TypeError: decrypt() missing 2 required positional arguments: 'secretKey' and 'enc'

In [ ]:
he.SerializationUtils.writeCiphertext(ctx1, "./ctx1.dat")
he.SerializationUtils.writeCiphertext(ctx1_1, "./ctx1_1.dat")
he.SerializationUtils.writeCiphertext(ctx2, "./ctx2.dat")

In [17]:
scheme2.multAndEqual(ctx1, ctx2)

In [18]:
out = decrypt(scheme, secretKey, ctx1)
print(out)

In [20]:
scheme2.loadLeftRotKey(1)

In [23]:
scheme.leftRotateFastAndEqual(ctx1, 1)
scheme2.leftRotateFastAndEqual(ctx1_1, 1)

In [24]:
o1 = decrypt(scheme, secretKey, ctx1)
o11 = decrypt(scheme, secretKey, ctx1_1)

In [25]:
o1

array([ 1.99999933e+00,  2.99999789e+00,  3.99999962e+00, ...,
       -1.55216352e-06, -3.27959178e-06,  9.99994768e-01])

In [26]:
o11

array([ 2.00000077e+00,  3.00000069e+00,  4.00000071e+00, ...,
       -2.59000396e-06,  1.75775797e-06,  1.00000723e+00])

Encryption, Decryption and Rotation are working! 